In [17]:
import os, sys
from pathlib import Path
print("CWD:", os.getcwd())
print("sys.path[0]:", sys.path[0])


CWD: c:\Users\amanbatra\Chicken-Disease-Classification
sys.path[0]: C:\Users\amanbatra\AppData\Local\Programs\Python\Python313\python313.zip


In [18]:
from Chicken_Disease_Classifier.constants import CONFIG_FILE_PATH
from Chicken_Disease_Classifier.utils.common import read_yaml

print("CONFIG_FILE_PATH:", CONFIG_FILE_PATH)
cfg = read_yaml(CONFIG_FILE_PATH)
print("Top-level keys:", list(cfg.keys()))
print("Has 'prepare_callbacks'?", "prepare_callbacks" in cfg)


CONFIG_FILE_PATH: config\config.yaml
[2025-10-23 04:06:06,033: INFO: common: yaml file: config\config.yaml loaded successfully]
Top-level keys: ['artifacts_root', 'data_ingestion', 'prepare_base_model', 'training']
Has 'prepare_callbacks'? False


In [2]:

%pwd

'c:\\Users\\amanbatra\\Chicken-Disease-Classification\\research'

In [3]:

os.chdir("../")

%pwd


'c:\\Users\\amanbatra\\Chicken-Disease-Classification'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [8]:

from Chicken_Disease_Classifier.constants import *
from Chicken_Disease_Classifier.utils.common import read_yaml, create_directories

In [13]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [14]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [19]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [23]:
from Chicken_Disease_Classifier.config.configuration import ConfigurationManager
cm = ConfigurationManager()
cb = cm.get_prepare_callback_config()
cb


[2025-10-23 04:11:46,259: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-23 04:11:46,264: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-23 04:11:46,266: INFO: common: created directory at: artifacts]
[2025-10-23 04:11:46,268: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoints]
[2025-10-23 04:11:46,270: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_logs]


PrepareCallbacksConfig(root_dir=WindowsPath('artifacts/prepare_callbacks'), tensorboard_root_log_dir=WindowsPath('artifacts/prepare_callbacks/tensorboard_logs'), checkpoint_model_filepath=WindowsPath('artifacts/prepare_callbacks/checkpoints/model.ckpt'))

In [24]:
import time
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
f"tb_logs_at_{timestamp}"

'tb_logs_at_2025-10-23-04-14-07'